# Roll No:


*   210607 - Eman Ali Sarwar
*   210630 - Alyyan Ahmed


In [ ]:
from collections import defaultdict
import nltk
import math
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from collections import defaultdict
import nltk
from nltk.util import ngrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.probability import LidstoneProbDist

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Part A**

In [ ]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
    return data

twitter_data = load_data('//content/drive/MyDrive/NLP_Project/twitter.txt')
print(twitter_data[:500])  # Print first 500 characters


How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.
When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.
they've decided its more fun if I don't.
So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)
Words from a complete stranger! Made my birthday even better :)
First Cubs game ever! Wrigley field is gorgeous. This is perfect. Go Cubs Go!
i no! i ge


In [ ]:
def split_to_sentences(data):
    sentences = data.split('\n')
    return sentences

sentences = split_to_sentences(twitter_data)
print(sentences[:5])  # Print first 5 sentences


['How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.', "When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.", "they've decided its more fun if I don't.", 'So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)', 'Words from a complete stranger! Made my birthday even better :)']


In [ ]:
nltk.download('punkt')
# Function to tokenize sentences
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence.lower())  # Convert to lowercase during tokenization
        tokenized_sentences.append(tokens)
    return tokenized_sentences


tokenized_sentences = tokenize_sentences(sentences)
print(tokenized_sentences[:5])  # Print tokenized sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['how', 'are', 'you', '?', 'btw', 'thanks', 'for', 'the', 'rt', '.', 'you', 'gon', 'na', 'be', 'in', 'dc', 'anytime', 'soon', '?', 'love', 'to', 'see', 'you', '.', 'been', 'way', ',', 'way', 'too', 'long', '.'], ['when', 'you', 'meet', 'someone', 'special', '...', 'you', "'ll", 'know', '.', 'your', 'heart', 'will', 'beat', 'more', 'rapidly', 'and', 'you', "'ll", 'smile', 'for', 'no', 'reason', '.'], ['they', "'ve", 'decided', 'its', 'more', 'fun', 'if', 'i', 'do', "n't", '.'], ['so', 'tired', 'd', ';', 'played', 'lazer', 'tag', '&', 'ran', 'a', 'lot', 'd', ';', 'ughh', 'going', 'to', 'sleep', 'like', 'in', '5', 'minutes', ';', ')'], ['words', 'from', 'a', 'complete', 'stranger', '!', 'made', 'my', 'birthday', 'even', 'better', ':', ')']]


In [ ]:
def split_train_test(tokenized_sentences, test_size=0.2):
    train_data, test_data = train_test_split(tokenized_sentences, test_size=test_size, random_state=42)
    return train_data, test_data

train_data, test_data = split_train_test(tokenized_sentences)
print(len(train_data), len(test_data))  # Print sizes of train and test data


38369 9593


In [ ]:
def count_words(tokenized_sentences):
    word_counts = defaultdict(int)
    for sentence in tokenized_sentences:
        for token in sentence:
            word_counts[token] += 1
    return word_counts

word_counts = count_words(train_data)
print(list(word_counts.items())[:10])  # Print first 10 items from word_counts


[('where', 461), ('are', 2699), ('you', 9818), ('skiing', 4), ('?', 6793), ('that', 4444), ("'s", 4888), ('right', 754), ('will', 1547), ('!', 20865)]


In [ ]:
def get_words_with_n_frequency(word_counts, n):
    frequent_words = {word for word, count in word_counts.items() if count >= n}
    return frequent_words

threshold = 1
frequent_words = get_words_with_n_frequency(word_counts, threshold)
print(f'Total frequent words with frequency >= {threshold}: {len(frequent_words)}')


Total frequent words with frequency >= 1: 36343


In [ ]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences:
        replaced_sentence = [token if token in vocabulary else unknown_token for token in sentence]
        replaced_tokenized_sentences.append(replaced_sentence)
    return replaced_tokenized_sentences

vocabulary = frequent_words
train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
print(train_data_replaced[:5])  # Print replaced training data sentences
print(test_data_replaced[:5])   # Print replaced test data sentences


[['where', 'are', 'you', 'skiing', '?'], ['that', "'s", 'right', 'you', 'will', '!', ':', ')', 'rt', ':', 'silly', 'class', 'u', 'preventd', 'me', 'from', 'winning', 'tt', '.', 'but', 'ill', 'get', 'them', 'next', 'time'], ['#', 'nowplaying', '``', 'zellmar', "'s", 'delight', "''", ',', 'sonny', 'clark'], ['looking', 'forward', 'to', 'it', '!', 'xo'], ['fuck', 'you', 'romantic', 'comedies', '.', 'fuck', '.', 'you', '.']]
[['the', 'industry', 'keeps', 'seeking', 'ways', 'to', '<unk>', 'the', 'fickle', 'power', 'of', 'pure', 'creativity', '.', 'but', 'nothing', "'s", 'beat', 'it', 'yet', '.'], ['perfect', 'for', 'easter', '!', 'thanks', '...', 'd'], ['we', "'re", 'about', 'to', 'blow', 'your', 'feed', 'up', ',', 'brace', 'yourselves', '...', '<', '3'], ['rt', 'if', 'you', "'re", 'happy', 'for', 'austin', 'because', 'he', "'s", 'living', 'his', 'dream', '<', '3'], ['ha', ',', 'eric', ',', 'you', '&', 'joyce', 'r', 'the', 'rock', 'stars', '!', 'actually', ',', 'i', 'think', 'i', "'ve", 'se

In [ ]:
# Function to preprocess data
def preprocess_data(train_data, test_data, threshold):
    word_counts = count_words(train_data)
    vocabulary = get_words_with_n_frequency(word_counts, threshold)
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
    return train_data_replaced, test_data_replaced, vocabulary


# Preprocess data
threshold = 1
train_data_final, test_data_final, vocabulary_final = preprocess_data(train_data, test_data, threshold)

print(train_data_final[:5])  # Print final training data sentences
print(test_data_final[:5])   # Print final test data sentences
print(f'Total vocabulary size: {len(vocabulary_final)}')
print(vocabulary_final)

[['where', 'are', 'you', 'skiing', '?'], ['that', "'s", 'right', 'you', 'will', '!', ':', ')', 'rt', ':', 'silly', 'class', 'u', 'preventd', 'me', 'from', 'winning', 'tt', '.', 'but', 'ill', 'get', 'them', 'next', 'time'], ['#', 'nowplaying', '``', 'zellmar', "'s", 'delight', "''", ',', 'sonny', 'clark'], ['looking', 'forward', 'to', 'it', '!', 'xo'], ['fuck', 'you', 'romantic', 'comedies', '.', 'fuck', '.', 'you', '.']]
[['the', 'industry', 'keeps', 'seeking', 'ways', 'to', '<unk>', 'the', 'fickle', 'power', 'of', 'pure', 'creativity', '.', 'but', 'nothing', "'s", 'beat', 'it', 'yet', '.'], ['perfect', 'for', 'easter', '!', 'thanks', '...', 'd'], ['we', "'re", 'about', 'to', 'blow', 'your', 'feed', 'up', ',', 'brace', 'yourselves', '...', '<', '3'], ['rt', 'if', 'you', "'re", 'happy', 'for', 'austin', 'because', 'he', "'s", 'living', 'his', 'dream', '<', '3'], ['ha', ',', 'eric', ',', 'you', '&', 'joyce', 'r', 'the', 'rock', 'stars', '!', 'actually', ',', 'i', 'think', 'i', "'ve", 'se

# **Part B**

In [ ]:
from collections import defaultdict, Counter

# Function to count n-grams in the data
def count_ngrams(data, n):
    ngrams_counts = defaultdict(Counter)
    for sentence in data:
        ngrams_in_sentence = ngrams(sentence, n, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')
        for ngram in ngrams_in_sentence:
            history, word = ngram[:-1], ngram[-1]
            ngrams_counts[history][word] += 1
    return ngrams_counts

# Function to create n-gram models for 1 to 6-grams
def get_ngram_models(data):
    return [count_ngrams(data, n) for n in range(1, 7)]

ngram_models = get_ngram_models(train_data_final)

In [ ]:
# Function to estimate probability of a word given a history
def estimate_probability(word, history, ngram_counts, nplus1gram_counts, vocabulary_size, k=1.0):
    history = tuple(history)
    nplus1gram = history + (word,)

    count_history = ngram_counts[history][word] if history in ngram_counts else 0
    count_nplus1gram = nplus1gram_counts[nplus1gram] if nplus1gram in nplus1gram_counts else 0

    probability = (count_nplus1gram + k) / (count_history + k * vocabulary_size)
    return probability

# Example usage for trigrams (n=3)
n = 3
ngram_counts = ngram_models[n-1]  # Trigram counts
nplus1gram_counts = ngram_models[n]  # 4-gram counts (trigram + 1)
vocabulary_final = set(word for sentence in train_data_final for word in sentence)
vocabulary_final.add('<e>')  # Include end token in vocabulary

word = 'night'
previous_ngram = ['good']
probability = estimate_probability(word, previous_ngram, ngram_counts, nplus1gram_counts, len(vocabulary_final))
print(f'Probability of "{word}" given "{previous_ngram}": {probability:.10f}')


Probability of "night" given "['good']": 0.0000275149


In [ ]:
# Create count and probability matrices for 1 to 6-grams
def create_count_and_probability_matrices(ngram_models, vocabulary):
    count_matrices = []
    probability_matrices = []

    for n in range(1, 7):
        if n < len(ngram_models):
            ngram_counts = ngram_models[n-1]
            nplus1gram_counts = ngram_models[n]

            # Initialize count and probability matrices
            count_matrix = defaultdict(lambda: defaultdict(int))
            probability_matrix = defaultdict(lambda: defaultdict(float))

            # Calculate vocabulary size
            vocabulary_size = len(vocabulary)

            # Iterate over all ngrams in ngram_counts
            for history, words_counts in ngram_counts.items():
                total_count = sum(words_counts.values())
                for word, count in words_counts.items():
                    count_matrix[history][word] = count
                    probability_matrix[history][word] = (count + 1.0) / (total_count + vocabulary_size)

            count_matrices.append(count_matrix)
            probability_matrices.append(probability_matrix)
        else:
            count_matrices.append({})
            probability_matrices.append({})

    return count_matrices, probability_matrices

# Example usage to create matrices
vocabulary_final = set(word for sentence in train_data_final for word in sentence)
count_matrices, probability_matrices = create_count_and_probability_matrices(ngram_models, vocabulary_final)

In [ ]:
# Print count and probability matrices for trigrams (n=3)
print("\nCount Matrix for Trigrams:")
for history, word_counts in count_matrices[2].items():
    print(f'History: {history}')
    for word, count in word_counts.items():
        print(f'  {word}: {count}')

Streaming output truncated to the last 5000 lines.
  still: 1
History: ('public', 'still')
  loves: 1
History: ('loves', 'him')
  ,: 1
History: ('much', 'worse')
  the: 1
History: ('worse', 'the')
  books: 1
History: ('too', 'vulnerable')
  </s>: 1
History: ('vulnerable', '</s>')
  </s>: 1
History: ('to', 'butt-tweet')
  .: 1
History: ('butt-tweet', '.')
  </s>: 1
History: ('i', 'thoroughly')
  enjoyed: 1
History: ('thoroughly', 'enjoyed')
  that: 1
History: ('enjoyed', 'that')
  .: 1
History: ('midwest', 'tonight')
  .: 1
History: ('people', 'mad')
  cause: 1
History: ('lakers', 'garbage')
  .: 1
History: ('do', 'yall')
  play: 1
  get: 1
History: ('yall', 'play')
  for: 1
History: ('you', 'cryin')
  for: 1
History: ('cryin', 'for')
  they: 1
History: ('aint', 'payin')
  yall: 1
History: ('payin', 'yall')
  </s>: 1
History: ('yall', '</s>')
  </s>: 1
History: ('<s>', '9:50')
  -: 1
History: ('9:50', '-')
  getting: 1
History: ('-', 'getting')
  my: 1
History: ('mask', '&')
  travel: 1

In [ ]:
print("\nProbability Matrix for Trigrams:")
for history, word_probs in probability_matrices[2].items():
    print(f'History: {history}')
    for word, prob in word_probs.items():
        print(f'  {word}: {prob:.6f}')

Streaming output truncated to the last 5000 lines.
  still: 0.000055
History: ('public', 'still')
  loves: 0.000055
History: ('loves', 'him')
  ,: 0.000055
History: ('much', 'worse')
  the: 0.000055
History: ('worse', 'the')
  books: 0.000055
History: ('too', 'vulnerable')
  </s>: 0.000055
History: ('vulnerable', '</s>')
  </s>: 0.000055
History: ('to', 'butt-tweet')
  .: 0.000055
History: ('butt-tweet', '.')
  </s>: 0.000055
History: ('i', 'thoroughly')
  enjoyed: 0.000055
History: ('thoroughly', 'enjoyed')
  that: 0.000055
History: ('enjoyed', 'that')
  .: 0.000055
History: ('midwest', 'tonight')
  .: 0.000055
History: ('people', 'mad')
  cause: 0.000055
History: ('lakers', 'garbage')
  .: 0.000055
History: ('do', 'yall')
  play: 0.000055
  get: 0.000055
History: ('yall', 'play')
  for: 0.000055
History: ('you', 'cryin')
  for: 0.000055
History: ('cryin', 'for')
  they: 0.000055
History: ('aint', 'payin')
  yall: 0.000055
History: ('payin', 'yall')
  </s>: 0.000055
History: ('yall', 

# **Part C**

In [ ]:
import math

def calculate_perplexity(test_data, ngram_models, vocabulary_size, k=1.0):
    N = 0  # Total number of words in the test data
    log_prob_sum = 0.0  # Sum of log probabilities (avoid underflow with float)

    for sentence in test_data:
        N += len(sentence)  # Update N efficiently for each sentence

        for n in range(1, len(ngram_models) + 1):
            for i in range(n - 1, len(sentence)):
                ngram = tuple(sentence[i - n + 1:i])
                word = sentence[i]

                if n <= len(ngram_models):
                    ngram_counts = ngram_models[n - 1]
                else:
                    ngram_counts = defaultdict(int)

                probability = estimate_probability(word, ngram, ngram_counts, vocabulary_size, k)

                if probability > 0.0:
                    log_prob_sum += -math.log(probability, 2)
                else:
                    print(f"Warning: Zero probability encountered for n-gram {ngram} and word {word}. Consider adjusting smoothing or handling techniques.")

    if N > 0:
        perplexity = 2 ** (log_prob_sum / N)
    else:
        perplexity = float('inf')  # Set perplexity to infinity for empty test data

    return perplexity


# **Part D**

In [ ]:
def suggest_next_word(previous_tokens, ngram_models, vocabulary, k=1.0):
    suggestions = defaultdict(float)
    n = len(previous_tokens) + 1

    if n <= len(ngram_models):
        ngram_counts = ngram_models[n - 1]
        nplus1gram_counts = ngram_models[n] if n < len(ngram_models) else defaultdict(int)
    else:
        ngram_counts = defaultdict(int)
        nplus1gram_counts = defaultdict(int)

    for word in vocabulary:
        probability = estimate_probability(word, previous_tokens, ngram_counts, nplus1gram_counts, len(vocabulary), k)
        suggestions[word] += probability

    if suggestions:
        sorted_suggestions = sorted(suggestions.items(), key=lambda item: item[1], reverse=True)
        return sorted_suggestions[:5]
    else:
        return []

# Function to sort suggestions based on probability
def sort_suggestions(suggestions):
    return sorted(suggestions, key=lambda x: x[1], reverse=True)


# Auto-complete function
def auto_complete(input_sentence, ngram_models, vocabulary, start_with=''):
    input_words = input_sentence.lower().split()
    n = len(input_words)
    suggestions = defaultdict(list)

    for i in range(n):
        history = input_words[max(0, i - 6):i]  # Adjust the range to include up to 6 previous words
        for model_n, ngram_counts in enumerate(ngram_models):
            if len(history) >= model_n:
                history_tuple = tuple(history[-model_n:])
                for word in vocabulary:
                    probability = estimate_probability(word, history_tuple, ngram_counts[model_n], ngram_counts[model_n + 1], len(vocabulary), k=1.0)
                    suggestions[model_n + 1].append((word, probability))

    filtered_suggestions = defaultdict(list)
    for model_n, model_suggestions in suggestions.items():
        if start_with:
            model_suggestions = [suggestion for suggestion in model_suggestions if suggestion[0].startswith(start_with)]
        model_suggestions = sort_suggestions(model_suggestions)[:5]  # Limit to top 5 suggestions
        filtered_suggestions[model_n] = model_suggestions

    return filtered_suggestions



In [ ]:
while True:
        input_sentence = input("\nEnter a sentence or word (type 'exit' to quit): ").strip()
        if input_sentence.lower() == 'exit':
            break

        start_with = input("Filter suggestions starting with (leave blank for all): ").strip()
        suggestions = auto_complete(input_sentence, ngram_models, vocabulary_final, start_with=start_with)

        print('\nAll suggestions:')
        for n, suggestion in suggestions.items():
            if suggestion:
                print(f'{n}-gram model: {suggestion}')
            else:
                print(f'{n}-gram model: No suggestions')


Enter a sentence or word (type 'exit' to quit): i like
Filter suggestions starting with (leave blank for all): app

All suggestions:
1-gram model: [('appointeeship', 2.751561511157582e-05), ('apple.com', 2.751561511157582e-05), ('appreciative', 2.751561511157582e-05), ('applying', 2.751561511157582e-05), ('appalled', 2.751561511157582e-05)]
2-gram model: [('appointeeship', 2.751561511157582e-05), ('apple.com', 2.751561511157582e-05), ('appreciative', 2.751561511157582e-05), ('applying', 2.751561511157582e-05), ('appalled', 2.751561511157582e-05)]

Enter a sentence or word (type 'exit' to quit): do you live
Filter suggestions starting with (leave blank for all): nea

All suggestions:
1-gram model: [('neara11', 2.751561511157582e-05), ('nearer', 2.751561511157582e-05), ('near-beer', 2.751561511157582e-05), ('nearby', 2.751561511157582e-05), ('near', 2.751561511157582e-05)]
2-gram model: [('neara11', 2.751561511157582e-05), ('nearer', 2.751561511157582e-05), ('near-beer', 2.7515615111575